**Edit Distance**

In [1]:
def editDistance(str1, str2, m, n):
    if m == 0:
        return n
    if n == 0:
        return m

    if str1[m-1] == str2[n-1]:
        return editDistance(str1, str2, m-1, n-1)

    return 1 + min(editDistance(str1, str2, m, n-1),    # Insert
                   editDistance(str1, str2, m-1, n),    # Remove
                   editDistance(str1, str2, m-1, n-1)    # Replace
                   )

In [2]:
str1=input("Enter First String:")
str2=input("Enter Second String:")
print(editDistance(str1, str2, len(str1), len(str2)))

Enter First String:hello
Enter Second String:good
5


**Poem Generation**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.utils as ku
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [12]:
data = open('/content/poem.txt', encoding="utf8").read()
corpus = data.lower().split("\n")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index)

print("Total Words:", total_words)

Total Words: 353


**Text to Embedding**

In [13]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
label = ku.to_categorical(label, num_classes=total_words+1)

**Bi-directional LSTM Model**

In [14]:
model = Sequential()
model.add(Embedding(total_words+1, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words+1/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words+1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 9, 100)            35400     
                                                                 
 bidirectional_1 (Bidirecti  (None, 9, 300)            301200    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 9, 300)            0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               160400    
                                                                 
 dense_2 (Dense)             (None, 353)               35653     
                                                                 
 dense_3 (Dense)             (None, 354)               125316    
                                                      

In [15]:
history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
22/22 [==============================] - 9s 59ms/step - loss: 6.9734 - accuracy: 0.0579
Epoch 2/100
22/22 [==============================] - 1s 59ms/step - loss: 6.0891 - accuracy: 0.0507
Epoch 3/100
22/22 [==============================] - 1s 58ms/step - loss: 5.6685 - accuracy: 0.0651
Epoch 4/100
22/22 [==============================] - 1s 59ms/step - loss: 5.4356 - accuracy: 0.0622
Epoch 5/100
22/22 [==============================] - 2s 104ms/step - loss: 5.2331 - accuracy: 0.0810
Epoch 6/100
22/22 [==============================] - 2s 72ms/step - loss: 5.0562 - accuracy: 0.0854
Epoch 7/100
22/22 [==============================] - 1s 57ms/step - loss: 4.8999 - accuracy: 0.0941
Epoch 8/100
22/22 [==============================] - 1s 58ms/step - loss: 4.7606 - accuracy: 0.0970
Epoch 9/100
22/22 [==============================] - 1s 57ms/step - loss: 4.6112 - accuracy: 0.1071
Epoch 10/100
22/22 [==============================] - 1s 57ms/step - loss: 4.4988 - accuracy: 0.120

**Poem Generation**

In [11]:
seed_text = input("Enter first few words")
next_words = 50
ouptut_text = ""

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1,
      padding='pre')
    predicted = np.argmax(model.predict(token_list,
                                        verbose=0), axis=-1)
    output_word = ""

    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break

    seed_text += " " + output_word

print(seed_text)

Enter first few wordsthe prettiest boy i've ever seen
the prettiest boy i've ever seen whispers of our inner psalm flight true true dawn true dawn done in we our stand true true done done done done dawn dawn love stand true dawn done done done done dawn given scars we our tomb stand true true done done done dawn dawn love stand true true
